<a href="https://colab.research.google.com/github/predictions01/zabbix/blob/main/predictions_multi_macchina.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
 from google.colab import drive
 drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
pip install lime shap eli5

In [2]:
import pandas as pd
import numpy as np

In [6]:
%run drive/MyDrive/colab_Notebooks/librerie/AJ_draw.py
%run drive/MyDrive/colab_Notebooks/librerie/AJ_ML_eda.py
%run drive/MyDrive/colab_Notebooks/librerie/AJ_models_classifier.py

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.feature_selection.base module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.feature_selection. Anything that cannot be imported from sklearn.feature_selection is now part of the private API.
  warnings.warn(message, FutureWarning)


In [8]:
main_folder = 'drive/MyDrive/Dati/'
metrics = pd.read_pickle(main_folder+"zabbix_AJ")
tickets = pd.read_pickle(main_folder+"tickets_AJ")

In [52]:
num_hours = 6
hours_pre_event = 10
hour_post_event = 0

In [63]:
def single_dev_prep(df_metrics, df_tickets, dev):
  my_metrics = df_metrics[df_metrics.device == dev].set_index('timestamp')[['metric', 'value']]
  df = my_metrics.pivot(columns='metric', values='value').dropna(thresh=my_metrics.index.nunique() * .9, axis=1).interpolate(limit_direction='both')
  df['hour'] = df.index.hour
  df['day'] = df.index.dayofweek
  df['week'] = df.index.isocalendar().week
  
  # selezioni macchina
  my_tickets = df_tickets[df_tickets['device'] == dev].set_index('timestamp')
  
  #crei la colonna dei target e droppi tutte le altre
  col_my_tickets = my_tickets.columns.tolist()
  my_tickets['y'] = 1
  my_tickets = my_tickets.drop(col_my_tickets, axis=1).reset_index()

  # arrotondi i tickets all'ora piu vicina
  my_tickets['timestamp'] = my_tickets['timestamp'].dt.round(freq = '1h')
  df = df.reset_index()

  #mergi i target con il df
  df = pd.merge(df, my_tickets, on = 'timestamp', how = 'outer')
  df['y'] = df['y'].fillna(0)

  #creo un indice dei tempi a cui si presentano i tickets a cui aggiungo il primo e l'ultimo evento del df
  vet_tickets_times = my_tickets['timestamp'].sort_values().tolist()
  vet_tickets_times = [df.iloc[0]['timestamp']] + vet_tickets_times + [df.iloc[-1]['timestamp']]

  #creo una colonna con la data dell'ultimo ticket aperto
  df_last_event = pd.DataFrame()
  for i in range(len(vet_tickets_times)-1):
      df_temp = pd.DataFrame()
      df_temp = df[df['timestamp'] >= vet_tickets_times[i] ][ df['timestamp'] < vet_tickets_times[i+1]]
      df_temp['last_event_time'] = vet_tickets_times[i]
      df_last_event = pd.concat([df_last_event, df_temp])

  #converto la data dell'ultimo ticket in numero di ore dall'ultimo ticket
  df_last_event = pd.DataFrame(df_last_event.iloc[:]['last_event_time'])
  df_last_event = pd.concat([df_last_event, pd.DataFrame([df_last_event.iloc[df_last_event.shape[0] - 1]['last_event_time']], columns = ['last_event_time'])])
  df['last_event_time'] = df_last_event.values
  df['last_event_delay'] = ((df['timestamp'] - df['last_event_time'])/np.timedelta64(1,'h')).astype(int)
  df = df.drop('last_event_time', axis = 1)

  df.set_index('timestamp', inplace=True)
  
  #funzione per mettere il label del ticket anche ad date successive o precedenti
  def increase_events_calculator(df, hours, direction = 1):
      hours = hours + 1
      for i in range(1,hours):
          df['y'+str(i)] = df['y'].shift(i*direction)

      for i in range(1,hours):
          df['y'] = df['y']+df['y'+str(i)]

      df['y'] = df['y'].fillna(0)
      dropy = ['y'+str(i) for i in range(1,hours)]
      df = df.drop(dropy, axis = 1)
      return df

  #aumento il numenro di label che rappresentano l'apertura del ticket
  df = increase_events_calculator(df, hour_post_event)
  df = increase_events_calculator(df, hours_pre_event, direction = -1)

  # le funzioni sopra creano creano numeri superiori a 1, ritrasforno tutto quello che e' superiore a 1 in 1
  df['y'] = df['y'].replace([i for i in range(1,int(df['y'].max()+1))], 1).astype(int)

  #scopro quali sono le metriche che correlano tra se stesse piu del 95% e le droppo
  _, correlation_within_features = learning(df).correlation_matrix('y', corr_value_w_targhet = 0.05, plot_matr = 'no')
  df = df.drop(correlation_within_features, axis = 1)

  #delle metriche rimaste scopro quali correlano con il target meno del 5% e le droppo
  correlation_with_target, _ = learning(df).correlation_matrix('y', corr_value_w_targhet = 0.05, plot_matr = 'no')
  df = df.drop(correlation_with_target.index.tolist(), axis = 1)
  return df

In [64]:
single_dev_prep(metrics, tickets, 'SQLDBONL10').head()

Boolean Series key will be reindexed to match DataFrame index.


,CPU%,Delta [E:] 15d,Delta [F:] 15d,File read bytes per second,File write bytes per second,Free disk space on C:,Free disk space on E:,Free disk space on F:,Free disk space on G:,Free disk space on G: (percentage),Free memory,Number of processes,Processor load (1 min average),System uptime,Timeleft [C:] pfree 0% (1d),Timeleft [F:] pfree 0% (1d),Timeleft [G:] pfree 0% (1d),Total disk space on E:,hour,day,week,y,last_event_delay
timestamp,,,,,,,,,,,,,,,,,,,,,,,
2020-11-12 13:00:00+00:00,5.276303,0.0,0.0,6.469184e+06,59946.375508,1.226939e+10,5.471621e+10,4.409761e+10,9.053142e+10,76.651096,1.273476e+09,52.0,0.0,23317729.0,1.000000e+12,1.000000e+12,1.000000e+12,2.684323e+11,13,3,46,0,0
2020-11-12 14:00:00+00:00,4.406250,0.0,0.0,1.608646e+06,51281.405373,1.217099e+10,5.471621e+10,4.409761e+10,9.053142e+10,76.651096,1.854061e+09,52.0,0.0,23320429.0,1.987969e+07,1.000000e+12,1.000000e+12,2.684323e+11,14,3,46,0,1
2020-11-12 15:00:00+00:00,1.662213,0.0,0.0,7.489163e+06,69639.087352,1.226849e+10,5.471621e+10,4.409761e+10,9.053142e+10,76.651096,2.149604e+09,52.0,0.0,23324029.0,1.000000e+12,1.000000e+12,1.000000e+12,2.684323e+11,15,3,46,0,2
2020-11-12 16:00:00+00:00,1.193871,0.0,0.0,3.643592e+05,59687.724004,1.226800e+10,5.471621e+10,4.409761e+10,9.053142e+10,76.651096,2.152799e+09,52.0,0.0,23327629.0,1.000000e+12,1.000000e+12,1.000000e+12,2.684323e+11,16,3,46,0,3
2020-11-12 17:00:00+00:00,1.187423,0.0,0.0,4.111920e+03,72305.967078,1.226752e+10,5.471621e+10,4.409761e+10,9.053142e+10,76.651096,2.135912e+09,52.0,0.0,23331229.0,1.000000e+12,1.000000e+12,1.000000e+12,2.684323e+11,17,3,46,0,4


In [65]:
metrics_x_device = dict()
devices = metrics['device'].unique().tolist()
for dev in devices:
  metrics_x_device[dev] = single_dev_prep(metrics, tickets, dev)

Boolean Series key will be reindexed to match DataFrame index.
Boolean Series key will be reindexed to match DataFrame index.
Boolean Series key will be reindexed to match DataFrame index.
Boolean Series key will be reindexed to match DataFrame index.
Boolean Series key will be reindexed to match DataFrame index.
Boolean Series key will be reindexed to match DataFrame index.
Boolean Series key will be reindexed to match DataFrame index.
Boolean Series key will be reindexed to match DataFrame index.
Boolean Series key will be reindexed to match DataFrame index.
Boolean Series key will be reindexed to match DataFrame index.
Boolean Series key will be reindexed to match DataFrame index.
Boolean Series key will be reindexed to match DataFrame index.
Boolean Series key will be reindexed to match DataFrame index.
Boolean Series key will be reindexed to match DataFrame index.
Boolean Series key will be reindexed to match DataFrame index.
Boolean Series key will be reindexed to match DataFrame

In [96]:
list_cols = []
for dev in devices:
  list_cols.append(metrics_x_device[dev].columns.tolist())
  # print(metrics_x_device[dev].columns.tolist())

In [106]:
List_flat = []
for i in range(len(list_cols)):
  for j in range (len(list_cols[i])):
    List_flat.append(list_cols[i][j])
df_List_flat = pd.DataFrame(List_flat)

unique_list = df_List_flat[0].unique()

df_counter = pd.DataFrame()

element = unique_list[0]
for element in unique_list:
  count = 0
  for dev in list_cols:
    if element in dev:
      count = count + 1
  df_counter[element] = [count]
print(df_counter.T[0].sort_values()[-10:])


Free swap space            15
System uptime              15
Free memory                16
CPU%                       17
Connections established    18
Free disk space on C:      18
week                       19
day                        21
last_event_delay           26
y                          34
Name: 0, dtype: int64


In [ ]:
min_device_count = 0

# Select uncorrelated columns

In [ ]:
def drop_correlated_columns(dataset, threshold):
    col_corr = set() # Set of all the names of deleted columns
    corr_matrix = dataset.corr()
    for i in range(len(corr_matrix.columns)):
        for j in range(i):
            if (corr_matrix.iloc[i, j] >= threshold) and (corr_matrix.columns[j] not in col_corr):
                colname = corr_matrix.columns[i] # getting the name of column
                col_corr.add(colname)
                if colname in dataset.columns:
                    del dataset[colname] # deleting the column from the dataset
    return dataset

uncorrelated_columns = drop_correlated_columns(pd.pivot_table(metrics, index='timestamp', columns='metric', values='value'), .9).columns

my_metrics = metrics[metrics['metric'].isin(uncorrelated_columns)]

# Seleziono le metriche comuni ad almeno \<min_device_count\> device

In [ ]:
common_metrics = my_metrics.groupby('metric').device.nunique().sort_values(ascending=False).where(lambda x: x > min_device_count).dropna().index

# Calcolo il dataframe - limito alle colonne scelte - interpolo i valori interni

In [ ]:
df_start = my_metrics.pivot(index=['timestamp', 'device'], columns='metric', values='value')[common_metrics] \
    .interpolate(limit_area='inside')


# Calcolo le metriche con delay

In [ ]:
def compute_delayed_metrics(df, delay, num_columns=5):
    cols_to_keep = ['y', 'hours_last_event']
    regex = re.compile(r"\[|\]|<", re.IGNORECASE)
    df.columns = df.columns.map(lambda x: regex.sub("", x))
    # Considero le <num_columns> colonne più correlate alla y
    cols = df.dropna(thresh=len(df) * .9, axis=1).corr().y.dropna().sort_values()[-num_columns - 1 : -1].index
    # Calcolo i delay
    for col in cols:
        cols_to_keep.append(col)
        for delay_hours in range(1, delay + 1):
            colname = f"{col}-{delay_hours}H"
            df.loc[:, colname] = df[col].shift(delay_hours)
            cols_to_keep.append(colname)

    return df[cols_to_keep]

# Calcolo la y

In [ ]:
def compute_y(df):
    df['y'] = 0
    df['hours_last_event'] = np.NAN
    idx = pd.IndexSlice
    for ticket in tickets.itertuples():
        start = max(ticket.timestamp.replace(minute=0, second=0) - pd.Timedelta(target - 1, unit='H'), df.index.get_level_values('timestamp').min())
        # Create 'y' column
        df.loc[idx[start : ticket.timestamp.replace(minute=0, second=0), ticket.device], 'y'] = 1
        # Create 'hours_last_event' column
        df.loc[idx[ticket.timestamp.ceil('H'), ticket.device], 'hours_last_event'] = 0
    return df


# Calcolo il numero di ore dall'ultimo evento

In [ ]:
def compute_hours_last_event(df):
    idx = pd.IndexSlice
    for device in df.index.get_level_values('device').unique():
        # Se non c'è un ticket precedente, setto a -1
        first_ticket = df.loc[idx[:, device], :].reset_index('device').hours_last_event.eq(0).idxmax()
        df.loc[idx[: first_ticket, device], 'hours_last_event'] = -1
        # Calcolo il numero di ore dal ticket precedente
        df.loc[idx[first_ticket:, device], 'hours_last_event']
        df.loc[idx[first_ticket:, device], 'hours_last_event'] = \
            df.loc[idx[first_ticket:, device], 'hours_last_event'].groupby(df.loc[idx[first_ticket:, device], 'hours_last_event'].notnull().cumsum()).cumcount()
    return df

# Aggiungo informazione temporale

In [ ]:
def compute_temporal_features(df):
    df['hour'] = df.index.get_level_values('timestamp').hour
    df['day'] = df.index.get_level_values('timestamp').dayofweek
    df['week'] = pd.Int64Index(df.index.get_level_values('timestamp').isocalendar().week)
    return df

# Splitto in modo bilanciato - con 70% di eventi positivi nel training set

In [ ]:
from sklearn.model_selection import train_test_split

def balanced_train_test_split(df, training_size=.7):
    # Calcolo quant'è il 70% del numero di "1"
    num_tickets_training = int(df.y.sum() * training_size)

    # Faccio cumulative sum per considerare nel training set un numero di ticket = num_tickets_training
    cumsum = df.y.cumsum()

    # Conto quante righe ho che stanno dopo i primi <num_tickets_training> ticket
    num_val_tickets = cumsum[cumsum > num_tickets_training].shape[0]
    
    # Splitto con test_size fissato
    training_set, validation_set = train_test_split(df, test_size=num_val_tickets, shuffle=False)
    
    return training_set, validation_set

# Effettuo upsampling degli 1 - scommentare se necessario

In [ ]:
# df_upsampled = pd.DataFrame([])

# for device in df.device.unique():
#     upsample = df[df.device == device].resample('5T').asfreq().interpolate()
#     upsample = upsample[(upsample.index.minute == 0) | (upsample.y == 1)]
#     for int_field in ["hours_last_event", "hour", "day", "week"]:
#         upsample[int_field] = upsample[int_field].round()
    
#     df_upsampled = pd.concat([
#         df_upsampled,
#         upsample
#     ])
# df = df_upsampled

In [ ]:
from sklearn.metrics import plot_confusion_matrix, accuracy_score, precision_score, recall_score, f1_score

def compute_score(y_true, y_pred, delay, target):
    score = {
        'delay': delay,
        'target': target,
        'accuracy': accuracy_score(y_true, y_pred),
        'precision': precision_score(y_true, y_pred),
        'recall': recall_score(y_true, y_pred),
        'f1': f1_score(y_true, y_pred),
        'positive_accuracy': accuracy_score(y_true[y_true == 1], y_pred[y_true == 1]),
        'negative_accuracy': accuracy_score(y_true[y_true == 0], y_pred[y_true == 0]),
    }
    return score

In [ ]:
from xgboost import XGBClassifier
from sklearn.metrics import plot_confusion_matrix, f1_score

# pensare di usare un validation set

scores = []

for target in range(3, 13):
    for delay in range(1, 13):
        df = df_start.copy()
        df = compute_y(df)
        df = compute_hours_last_event(df)
        df = compute_delayed_metrics(df, delay)
        df = compute_temporal_features(df)
        df.reset_index('device', inplace=True)
        df['device'] = df.device.factorize()[0]
        
        training_set, test_set = balanced_train_test_split(df[df.hours_last_event >= 0], .7)
        model = XGBClassifier(n_estimators=500, eval_metric='logloss', use_label_encoder=False).fit(training_set.drop('y', axis=1), training_set.y)
        y_pred = model.predict(test_set.drop('y', axis=1))
        scores.append(compute_score(test_set.y, y_pred, delay, target))
        
pd.DataFrame(scores)

NameError: name 'balanced_train_test_split' is not defined

In [ ]:
# train test divisi per device
# grid search: lunghezza info passata e #ore target
# informazione temporale
# trasformare in serie stazionarie